# Code Failure Example - Cantilevered Hyperelastic Beam

This tutorial starts with the previous example (in b-mesh_refinement.ipynb), except with a longer beam length, and shows how using a coarse mesh can cause the solver to fail.

First, we import dependencies and setup the simulation domain.

Here we extend the beam from a length of 20.0 to 40.0 and mesh with the same number of elements as in the previous example, [20, 5, 5]. This results in a much coarser mesh along the length of the beam.

In [121]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 40.0
nx = 20
ny = 2
nz = 2
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [nx, ny, nz], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

[2025-04-26 10:01:34.642] [info] Extract basic topology: 640->640
[2025-04-26 10:01:34.642] [info] Build local dual graph
[2025-04-26 10:01:34.642] [info] Build local part of mesh dual graph (mixed)
[2025-04-26 10:01:34.643] [info] GPS pseudo-diameter:(22) 79-0
[2025-04-26 10:01:34.643] [info] Create topology (single cell type)
[2025-04-26 10:01:34.643] [info] Create topology (generalised)
[2025-04-26 10:01:34.643] [info] Computing communication graph edges (using NBX algorithm). Number of input edges: 1
[2025-04-26 10:01:34.643] [info] Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
[2025-04-26 10:01:34.643] [info] Compute ghost indices
[2025-04-26 10:01:34.643] [info] Computing communication graph edges (using PCX algorithm). Number of input edges: 0
[2025-04-26 10:01:34.643] [info] Finished graph edge discovery using PCX algorithm. Number of discovered edges 0
[2025-04-26 10:01:34.643] [info] Computing communication graph edges (using NBX algorithm). 

To simplify boundary condition application, we next create functions to select the left and right groups of facets.

In [122]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

[2025-04-26 10:01:34.655] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 10:01:34.655] [info] Computing mesh connectivity 2-3 from transpose.
[2025-04-26 10:01:34.655] [info] Requesting connectivity (2, 0) - (0, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (2, 0) - (2, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (0, 0) - (3, 0)
[2025-04-26 10:01:34.655] [info] Computing mesh connectivity 0-3 from transpose.
[2025-04-26 10:01:34.655] [info] Requesting connectivity (3, 0) - (0, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (2, 0) - (0, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (2, 0) - (2, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (0, 0) - (3, 0)
[2025-04-26 10:01:34.655] [info] Requesting connectivity (3, 0) - (0, 0)


We can then apply the left side boundary conditions (fixed):

In [123]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force and traction, as well as test and solution functions.

In [124]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

v = ufl.TestFunction(V)
u = fem.Function(V)

Now we specify the kinematic properties:

In [125]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

...as well the hyperelasticity model parameters:

In [126]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

Define the weak form with traction integral over all facets with value 2.
 We set the quadrature degree for the integrals to 4, and use Dolfinx's nonlinearproblem solver.

In [127]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

problem = NonlinearProblem(F, u, bcs)

[2025-04-26 10:01:34.694] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 10:01:34.694] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-04-26 10:01:34.694] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 10:01:34.694] [info] Requesting connectivity (3, 0) - (2, 0)
[2025-04-26 10:01:34.698] [info] Requesting connectivity (2, 0) - (3, 0)
[2025-04-26 10:01:34.698] [info] Requesting connectivity (3, 0) - (2, 0)


Next we customize a newton solver:

In [128]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

[2025-04-26 10:01:34.705] [info] Column ghost size increased from 0 to 0


...and create a function to plot the deformation results at each time step:

In [129]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("c-deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

[2025-04-26 10:01:37.741] [info] Checking required entities per dimension
[2025-04-26 10:01:37.742] [info] Cell type: 0 dofmap: 80x27
[2025-04-26 10:01:37.742] [info] Global index computation
[2025-04-26 10:01:37.742] [info] Got 4 index_maps
[2025-04-26 10:01:37.742] [info] Get global indices


Finally, we solve the problem over several time steps, updating the z-component of the traction.

In [130]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

[2025-04-26 10:01:37.807] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 10:01:37.913] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 10:01:37.955] [info] Newton iteration 2: r (abs) = 405.9626629200634 (tol = 1e-08), r (rel) = 0.674755683934218 (tol = 1e-08)
[2025-04-26 10:01:38.005] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 10:01:38.047] [info] Newton iteration 3: r (abs) = 224.041069026844 (tol = 1e-08), r (rel) = 0.3723814985180753 (tol = 1e-08)
[2025-04-26 10:01:38.094] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 10:01:38.137] [info] Newton iteration 4: r (abs) = 250.76735318125316 (tol = 1e-08), r (rel) = 0.4168035939243702 (tol = 1e-08)
[2025-04-26 10:01:38.185] [info] PETSc Krylov solver starting to solve system.
[2025-04-26 10:01:38.226] [info] Newton iteration 5: r (abs) = nan (tol = 1e-08), r (rel) = nan (tol = 1e-08)
[2025-04-26 10:01:38.275] [info] PETSc Krylov solver starting to solve system.


RuntimeError: Newton solver did not converge because maximum number of iterations reached

<img src="b-deformation.png" alt="Beam Deformation" width="500"/>

As we can see, the newton solver failed to converge due to the coarse mesh size and long, slender geometry.